## Libraries Used

In [ ]:
# %pip install pandas matplotlib seaborn  nltk textblob vaderSentiment wordcloud

# Data Handling
import pandas as pd
import numpy as np

# Text Processing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Display Formatting
from IPython.display import display

## Dataset extraction and organization
### Load and combine all datasets

In [10]:
import pandas as pd # for dataframes
from IPython.display import display # for display

# Load datasets
dataset1 = pd.read_csv('data/amazon_cells_labelled.txt', delimiter='\t', header=None, names=['Review', 'Sentiment'])
dataset2 = pd.read_csv('data/imdb_labelled.txt', delimiter='\t', header=None, names=['Review', 'Sentiment'])
dataset3 = pd.read_csv('data/yelp_labelled.txt', delimiter='\t', header=None, names=['Review', 'Sentiment'])

# Combine datasets
combined_dataset = pd.concat([dataset1, dataset2, dataset3], ignore_index=True)

# Ensure correct column ordering
combined_dataset = combined_dataset[['Review', 'Sentiment']]

# Convert Sentiment column to a centered string format 
combined_dataset["Sentiment"] = combined_dataset["Sentiment"].astype(str).apply(lambda x: f"{x:^5}")

# Display first 10 rows
display(combined_dataset.head(10)) 


,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
5,I have to jiggle the plug to get it to line up...,0
6,If you have several dozen or several hundred c...,0
7,If you are Razr owner...you must have this!,1
8,"Needless to say, I wasted my money.",0
9,What a waste of money and time!.,0


## Data Cleaning and Preparation
### Steps: 
#### 1. Remove duplicates.
#### 2.Handle missing values.
#### 3.Lowercase text for consistency.
#### 4. Remove punctuation and stopwords.

## Exploratory Data Analysis
### Visualizing and analyzing sentiment distribution.

### Distribution

### Word Cloud for Most Frequent Word

## Model Selection
### Vader

### Logisitic Regression 

## Model Analysis 
### Vader vs Logistic Regression Results

## Conclusion